In [1]:
from data_utils import SIFT50MDataset
import os
from datasets import load_dataset, Dataset

sift_dataset = load_dataset(
    'amazon-agi/SIFT-50M',
    #name='closed_ended_comparison',
    name = 'closed_ended_content_level',
    split='train',
    trust_remote_code=True
)

sift_dataset_test = load_dataset(
    'amazon-agi/SIFT-50M',
    #name='closed_ended_comparison',
    name = 'closed_ended_content_level',
    split='EvalSIFT',
    trust_remote_code=True
) 

# Define the allowed data sources
allowed_values = ["common_voice_de", "common_voice_en", "vctk_en"]

# Define a filter function
def filter_data_source(example):
    return example["data_source"] in allowed_values

filtered_sift_data = sift_dataset.filter(filter_data_source)
filtered_sift_data_test = sift_dataset_test.filter(filter_data_source)

#df = sift_dataset.select(range(1000)).to_pandas()
print(len(filtered_sift_data))
print(len(filtered_sift_data_test))
#filtered_df = df[df["data_source"].isin(allowed_values)]

# Count number of entries per value
#counts = filtered_df["data_source"].value_counts()
#print(counts)
#sift_data = Dataset.from_pandas(filtered_df)
# Define the base datasets paths (replace with your actual paths)

'''print('Training set')
counts = filtered_sift_data.to_pandas()["data_source"].value_counts()
print(counts)
print('Evaluation set')
counts = filtered_sift_data_eval.to_pandas()["data_source"].value_counts()'''


Resolving data files:   0%|          | 0/1780 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1450 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1780 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1450 [00:00<?, ?it/s]

3966476
601


'print(\'Training set\')\ncounts = filtered_sift_data.to_pandas()["data_source"].value_counts()\nprint(counts)\nprint(\'Evaluation set\')\ncounts = filtered_sift_data_eval.to_pandas()["data_source"].value_counts()'

In [2]:
base_datasets_root = "/home/jovyan/.cache/huggingface/datasets"
base_datasets_paths = {
    "common_voice_de": None, # No longer needs a path, handled by load_dataset
    #"multilingual_librispeech_de": None, # No longer needs a path, handled by load_dataset
    "common_voice_en": None, # No longer needs a path, handled by load_dataset
    "vctk_en": "./vctk_corpus" # VCTK still needs a root path for torchaudio
}

In [3]:
sift_iterable_dataset_eval = SIFT50MDataset(sift_dataset=filtered_sift_data_test, base_datasets_paths=base_datasets_paths, test_set=True)

for x in sift_iterable_dataset_eval:
    print(x)
    break

{'id': 'common_voice_de_25543288', 'messages': [{'role': 'user', 'content': [{'text': None, 'audio_path': '/home/jovyan/.cache/huggingface/datasets/downloads/extracted/e2d0a7ef8aa3a7692bd12936cfc8b5ca687d58a4fedbd18ac6820dbe1aef6b8d/de_test_0/common_voice_de_25543288.mp3', 'type': 'audio'}, {'text': 'Ist das Audio für ein erwachsenes Publikum geeignet?', 'audio_path': None, 'type': 'text'}]}, {'role': 'assistant', 'content': [{'text': 'Ja, das Audio ist für ein erwachsenes Publikum geeignet.', 'audio_path': None, 'type': 'text'}]}], 'task': 'closed_ended_content_level', 'data_source': 'common_voice_de'}


In [4]:
sift_eval_list = list(sift_iterable_dataset_eval)
print(f"Dataset loaded. Total samples for evaluation: {len(sift_eval_list)}")
print("-" * 50)


print("Printing Ground Truth for the first 5 samples:")
for i in range(min(5, len(sift_eval_list))):
    sample = sift_eval_list[i]
    messages = sample.get("messages", [])
    
    if not messages:
        continue
    
    # Build ground truth (question + answer)
    ground_truth_texts = []
    for message in messages:
        for item in message.get("content", []):
            if item.get("type") == "text" and item.get("text"):
                ground_truth_texts.append(item["text"])
    
    gt_text = " ".join(ground_truth_texts).strip()

    print(f"\nSample {i+1} Ground Truth:")
    print(gt_text if gt_text else "<EMPTY>")
    print("-" * 50)

Dataset loaded. Total samples for evaluation: 601
--------------------------------------------------
Printing Ground Truth for the first 5 samples:

Sample 1 Ground Truth:
Ist das Audio für ein erwachsenes Publikum geeignet? Ja, das Audio ist für ein erwachsenes Publikum geeignet.
--------------------------------------------------

Sample 2 Ground Truth:
Um wie viele Uhr spielt die Frau in der Audiodatei über den Canal de la Marne au Rhin und die Bahnlinie Straßburg-Metz? Leider kann ich aus der Audio-Datei nicht hören, um wie viele Uhr die Frau spricht. Die Metadaten enthalten diese Information nicht.
--------------------------------------------------

Sample 3 Ground Truth:
Was steht auf dem Pannenstreifen, das der Sprecher im Audioclip erwähnt? Auf dem Pannenstreifen steht ein defekter LKW.
--------------------------------------------------

Sample 4 Ground Truth:
Ist die Stimme auf dem Audioclip weiblich oder männlich? Die Stimme ist männlich.
--------------------------------------

In [5]:
import os
import torch
import librosa
import pandas as pd
from transformers import AutoProcessor, Qwen2AudioForConditionalGeneration
import evaluate

# ----------------------
# Paths
# ----------------------
data_dir = "data/eval_audio_dataset"
snr_levels = ["clean", "noisy_0dB", "noisy_10dB", "noisy_20dB"]

# ----------------------
# Device (CPU only)
# ----------------------
device = "cpu"
print(f"Using device: {device}")

# ----------------------
# Load processor & model
# ----------------------
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
model = Qwen2AudioForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-Audio-7B-Instruct",
    trust_remote_code=True
).eval()  # keep on CPU

# ----------------------
# Load metrics
# ----------------------
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")
bertscore_metric = evaluate.load("bertscore")

# ----------------------
# Helper: clean generated text
# ----------------------
def clean_generated_text(generated_text):
    lines = generated_text.splitlines()
    cleaned_lines = []
    keep = False
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.lower().startswith("assistant"):
            keep = True
            continue
        if keep:
            cleaned_lines.append(line)
    return " ".join(cleaned_lines).strip()

# ----------------------
# Group files by sample index
# ----------------------
files_by_idx = {}
for fname in os.listdir(data_dir):
    if not fname.endswith(".wav"):
        continue
    parts = fname.split("_")
    if len(parts) < 3:
        continue
    idx = parts[1]
    files_by_idx.setdefault(idx, {})[fname] = os.path.join(data_dir, fname)

# ----------------------
# Collect results
# ----------------------
results = []
for idx, files in sorted(files_by_idx.items(), key=lambda x: int(x[0]))[:2]:

    # Retrieve dataset messages
    sample = next((s for j, s in enumerate(sift_iterable_dataset_eval) if j == int(idx)), None)
    if sample is None:
        continue

    messages = sample.get("messages", [])
    if not messages:
        continue

    # Build ground truth (question + answer)
    ground_truth_texts = []
    for message in messages:
        for item in message.get("content", []):
            if item.get("type") == "text" and item.get("text"):
                ground_truth_texts.append(item["text"])
    gt_text = " ".join(ground_truth_texts).strip()

    # Build conversation (with audio placeholders)
    conversation = []
    for message in messages:
        role = message.get("role", "user")
        content = []
        for item in message.get("content", []):
            if item.get("type") == "text" and item.get("text"):
                content.append({"type": "text", "text": item["text"]})
            elif item.get("type") == "audio":
                content.append({"type": "audio"})
        if content:
            conversation.append({"role": role, "content": content})

    print(f"\n=== Sample {idx} ===")
    print("Ground truth transcript:")
    print(gt_text if gt_text else "<EMPTY>")

    # ----------------------
    # Loop over SNR levels
    # ----------------------
    for snr in snr_levels:
        match = [f for f in files if snr in f]
        if not match:
            continue
        audio_path = files[match[0]]

        # Load audio
        audio_input, sr = librosa.load(audio_path, sr=16000)

        # Apply chat template
        text_prompt = processor.apply_chat_template(
            conversation, add_generation_prompt=True, tokenize=False
        )

        # Process audio + text
        inputs = processor(
            text=text_prompt,
            audio=audio_input,
            sampling_rate=16000,
            return_tensors="pt"
        )

        # Inference
        with torch.no_grad():
            generated_ids = model.generate(**inputs, max_new_tokens=128)

        # Decode and clean
        generated_text = processor.batch_decode(
            generated_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )[0]
        generated_text_clean = clean_generated_text(generated_text)

        print(f"[{snr}] {os.path.basename(audio_path)}")
        print(f"Generated text (cleaned): {generated_text_clean}")

        # ----------------------
        # Evaluation
        # ----------------------
        if gt_text:
            wer = wer_metric.compute(references=[gt_text], predictions=[generated_text_clean])
            cer = cer_metric.compute(references=[gt_text], predictions=[generated_text_clean])
            bertscore = bertscore_metric.compute(
                predictions=[generated_text_clean], references=[gt_text], lang="de"
            )
            results.append({
                "sample_idx": idx,
                "snr": snr,
                "ground_truth": gt_text,
                "generated_text": generated_text_clean,
                "wer": wer,
                "cer": cer,
                "bertscore_f1": bertscore["f1"][0]
            })
            print(f"WER: {wer:.3f}, CER: {cer:.3f}, BERTScore F1: {bertscore['f1'][0]:.3f}")
        else:
            results.append({
                "sample_idx": idx,
                "snr": snr,
                "ground_truth": "<EMPTY>",
                "generated_text": generated_text_clean,
                "wer": None,
                "cer": None,
                "bertscore_f1": None
            })
            print("No ground truth for evaluation.")

        print("-" * 50)

# ----------------------
# Convert to DataFrame
# ----------------------
df_results = pd.DataFrame(results)
print("\nFinal results table:")
print(df_results)



Using device: cpu


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]


=== Sample 0 ===
Ground truth transcript:
Ist das Audio für ein erwachsenes Publikum geeignet? Ja, das Audio ist für ein erwachsenes Publikum geeignet.
[clean] sample_0_clean.wav
Generated text (cleaned): Ja, das Audio ist für ein erwachsenes Publikum geeignet. Anzahl der Wörter: 23
WER: 0.706, CER: 0.523, BERTScore F1: 0.798
--------------------------------------------------
[noisy_0dB] sample_0_noisy_0dB.wav
Generated text (cleaned): Ja, das Audio ist für ein erwachsenes Publikum geeignet. Ist es in deutscher Sprache?
WER: 0.706, CER: 0.477, BERTScore F1: 0.842
--------------------------------------------------
[noisy_10dB] sample_0_noisy_10dB.wav
Generated text (cleaned): Ja, das Audio ist für ein erwachsenes Publikum geeignet. Können Sie bitte beschreiben, was im Audio gesagt wird?
WER: 0.706, CER: 0.495, BERTScore F1: 0.804
--------------------------------------------------
[noisy_20dB] sample_0_noisy_20dB.wav
Generated text (cleaned): Ja, das Audio ist für ein erwachsenes Publik